In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action="ignore")

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)

import matplotlib as mpl
mpl.rcParams["axes.unicode_minus"] = False

#import tensorflow as tf

import warnings
warnings.filterwarnings(action="ignore")

## 날씨

+ 데이터 교체
+ 불필요한 컬럼 정리 및 불쾌지수, 체감온도 추가
+ 불쾌지수와 체감온도 구간화

In [2]:
train_weather = pd.read_csv('data/train_weather.csv', encoding='euc-kr')
train_weather

,지점,지점명,일시,최저기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 상대습도(%)
0,192,진주,2016-02-01,-4.1,NaN,8.6,43.9
1,192,진주,2016-02-02,-9.1,NaN,4.2,47.5
2,192,진주,2016-02-03,-9.3,NaN,4.5,57.1
3,192,진주,2016-02-04,-7.7,NaN,7.0,53.4
4,192,진주,2016-02-05,-6.5,NaN,5.9,44.4
...,...,...,...,...,...,...,...
1817,192,진주,2021-01-22,5.1,1.2,2.3,97.5
1818,192,진주,2021-01-23,6.8,1.4,6.1,87.9
1819,192,진주,2021-01-24,3.0,0.0,7.9,70.4
1820,192,진주,2021-01-25,-1.2,NaN,6.2,75.8


In [3]:
# 지점, 지점명 삭제
train_weather = train_weather.drop('지점', axis=1)
train_weather = train_weather.drop('지점명', axis=1)

In [4]:
train_weather['일강수량(mm)'] = train_weather['일강수량(mm)'].replace({np.NaN:0})
train_weather

,일시,최저기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 상대습도(%)
0,2016-02-01,-4.1,0.0,8.6,43.9
1,2016-02-02,-9.1,0.0,4.2,47.5
2,2016-02-03,-9.3,0.0,4.5,57.1
3,2016-02-04,-7.7,0.0,7.0,53.4
4,2016-02-05,-6.5,0.0,5.9,44.4
...,...,...,...,...,...
1817,2021-01-22,5.1,1.2,2.3,97.5
1818,2021-01-23,6.8,1.4,6.1,87.9
1819,2021-01-24,3.0,0.0,7.9,70.4
1820,2021-01-25,-1.2,0.0,6.2,75.8


In [5]:
test_weather = pd.read_csv('data/test_weather.csv', encoding='euc-kr')
test_weather

,지점,지점명,일시,최저기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 상대습도(%)
0,192,진주,2021-01-27,-1.2,NaN,6.6,61.5
1,192,진주,2021-01-28,-3.0,NaN,14.6,58.1
2,192,진주,2021-01-29,-6.0,NaN,11.0,39.4
3,192,진주,2021-01-30,-9.2,NaN,13.0,69.4
4,192,진주,2021-01-31,-6.0,1.0,6.0,74.6
...,...,...,...,...,...,...,...
68,192,진주,2021-04-05,4.5,NaN,6.6,58.9
69,192,진주,2021-04-06,1.3,NaN,6.7,60.0
70,192,진주,2021-04-07,3.3,NaN,8.1,58.6
71,192,진주,2021-04-08,5.5,NaN,6.4,64.1


In [6]:
# 지점, 지점명 삭제
test_weather = test_weather.drop('지점', axis=1)
test_weather = test_weather.drop('지점명', axis=1)

In [7]:
test_weather['일강수량(mm)'] = test_weather['일강수량(mm)'].replace({np.NaN:0})
test_weather

,일시,최저기온(°C),일강수량(mm),최대 순간 풍속(m/s),평균 상대습도(%)
0,2021-01-27,-1.2,0.0,6.6,61.5
1,2021-01-28,-3.0,0.0,14.6,58.1
2,2021-01-29,-6.0,0.0,11.0,39.4
3,2021-01-30,-9.2,0.0,13.0,69.4
4,2021-01-31,-6.0,1.0,6.0,74.6
...,...,...,...,...,...
68,2021-04-05,4.5,0.0,6.6,58.9
69,2021-04-06,1.3,0.0,6.7,60.0
70,2021-04-07,3.3,0.0,8.1,58.6
71,2021-04-08,5.5,0.0,6.4,64.1


In [8]:
# 컬럼명 변경
train_weather.columns = ['일시', '기온', '강수량', '풍속', '습도']
test_weather.columns = ['일시', '기온', '강수량', '풍속', '습도']

In [9]:
train_weather

,일시,기온,강수량,풍속,습도
0,2016-02-01,-4.1,0.0,8.6,43.9
1,2016-02-02,-9.1,0.0,4.2,47.5
2,2016-02-03,-9.3,0.0,4.5,57.1
3,2016-02-04,-7.7,0.0,7.0,53.4
4,2016-02-05,-6.5,0.0,5.9,44.4
...,...,...,...,...,...
1817,2021-01-22,5.1,1.2,2.3,97.5
1818,2021-01-23,6.8,1.4,6.1,87.9
1819,2021-01-24,3.0,0.0,7.9,70.4
1820,2021-01-25,-1.2,0.0,6.2,75.8


In [10]:
# 결측치 확인

train_weather[train_weather[('습도')].isnull()]

,일시,기온,강수량,풍속,습도
712,2018-01-13,-12.1,0.0,7.9,NaN
991,2018-10-19,5.4,0.0,5.8,NaN


In [11]:
# 결측치 0으로 대체

train_weather['습도'] = train_weather['습도'].replace({np.NaN:0})
train_weather

,일시,기온,강수량,풍속,습도
0,2016-02-01,-4.1,0.0,8.6,43.9
1,2016-02-02,-9.1,0.0,4.2,47.5
2,2016-02-03,-9.3,0.0,4.5,57.1
3,2016-02-04,-7.7,0.0,7.0,53.4
4,2016-02-05,-6.5,0.0,5.9,44.4
...,...,...,...,...,...
1817,2021-01-22,5.1,1.2,2.3,97.5
1818,2021-01-23,6.8,1.4,6.1,87.9
1819,2021-01-24,3.0,0.0,7.9,70.4
1820,2021-01-25,-1.2,0.0,6.2,75.8


In [12]:
# 불쾌지수 계산 
def get_di(df:pd.DataFrame)->pd.DataFrame:
    df['불쾌지수'] = 9/5 * df['기온'] - 0.55 * (1-df['습도']/100) * (9/5 * df['기온'] - 26) + 32
    return df

train_weather = get_di(train_weather)
test_weather = get_di(test_weather)

In [13]:
# 체감온도 계산

def get_wct(df:pd.DataFrame)->pd.DataFrame:
    df['체감온도'] = 13.12 + 0.6215 * df['기온'] - 11.37 * (df['풍속'] ** 0.16) + 0.3965 * (df['풍속'] ** 0.16) * df['기온']
    return df

train_weather = get_wct(train_weather)
test_weather = get_wct(test_weather)

In [14]:
train_weather
train_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822 entries, 0 to 1821
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      1822 non-null   object 
 1   기온      1822 non-null   float64
 2   강수량     1822 non-null   float64
 3   풍속      1822 non-null   float64
 4   습도      1822 non-null   float64
 5   불쾌지수    1822 non-null   float64
 6   체감온도    1822 non-null   float64
dtypes: float64(6), object(1)
memory usage: 99.8+ KB


In [15]:
test_weather

,일시,기온,강수량,풍속,습도,불쾌지수,체감온도
0,2021-01-27,-1.2,0.0,6.6,61.5,35.802880,-3.646867
1,2021-01-28,-3.0,0.0,14.6,58.1,33.836130,-8.031683
2,2021-01-29,-6.0,0.0,11.0,39.4,33.465440,-10.787735
3,2021-01-30,-9.2,0.0,13.0,69.4,22.602848,-15.235758
4,2021-01-31,-6.0,1.0,6.0,74.6,26.340960,-8.922667
...,...,...,...,...,...,...,...
68,2021-04-05,4.5,0.0,6.6,58.9,44.146295,2.952329
69,2021-04-06,1.3,0.0,6.7,60.0,39.545200,-0.787847
70,2021-04-07,3.3,0.0,8.1,58.6,42.507662,1.109747
71,2021-04-08,5.5,0.0,6.4,64.1,45.078945,4.171122


In [16]:
# 불쾌지수 구간
# https://web.kma.go.kr/weather/lifenindustry/li_asset/HELP/basic/help_01_05.jsp

def int_di(x):
    if x >= 80:
        return 1
    elif (x >= 75) & (x < 80):
        return 2
    elif (x >= 68) & (x < 75):
        return 3
    elif x < 68:
        return 4
    
train_weather['불쾌지수구간'] = train_weather['불쾌지수'].apply(int_di)
test_weather['불쾌지수구간'] = test_weather['불쾌지수'].apply(int_di)

In [20]:
# 체감온도 구간
# https://www.weather.go.kr/weather/lifenindustry/li_asset/HELP/basic/JISU_HELP_v1.1.pdf

def int_wct(x):
    if x >= 37:
        return 1
    elif (x >= 34) & (x < 37):
        return 2
    elif (x >= 31) & (x < 34):
        return 3
    elif (x >= 29) & (x < 31):
        return 4
    elif x < 29:
        return 5
    
train_weather['체감온도구간'] = train_weather['체감온도'].apply(int_wct)
test_weather['체감온도구간'] = test_weather['체감온도'].apply(int_wct)

In [21]:
train_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822 entries, 0 to 1821
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      1822 non-null   object 
 1   기온      1822 non-null   float64
 2   강수량     1822 non-null   float64
 3   풍속      1822 non-null   float64
 4   습도      1822 non-null   float64
 5   불쾌지수    1822 non-null   float64
 6   체감온도    1822 non-null   float64
 7   불쾌지수구간  1822 non-null   int64  
 8   체감온도구간  1822 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 128.2+ KB


In [22]:
test_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      73 non-null     object 
 1   기온      73 non-null     float64
 2   강수량     73 non-null     float64
 3   풍속      73 non-null     float64
 4   습도      73 non-null     float64
 5   불쾌지수    73 non-null     float64
 6   체감온도    73 non-null     float64
 7   불쾌지수구간  73 non-null     int64  
 8   체감온도구간  73 non-null     int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 5.3+ KB


In [23]:
train_weather

,일시,기온,강수량,풍속,습도,불쾌지수,체감온도,불쾌지수구간,체감온도구간
0,2016-02-01,-4.1,0.0,8.6,43.9,34.919399,-7.764706,4,5
1,2016-02-02,-9.1,0.0,4.2,47.5,27.857225,-11.379865,4,5
2,2016-02-03,-9.3,0.0,4.5,57.1,25.344503,-11.814212,4,5
3,2016-02-04,-7.7,0.0,7.0,53.4,28.356118,-11.356778,4,5
4,2016-02-05,-6.5,0.0,5.9,44.4,31.828660,-9.447595,4,5
...,...,...,...,...,...,...,...,...,...
1817,2021-01-22,5.1,1.2,2.3,97.5,41.411275,5.609234,4,5
1818,2021-01-23,6.8,1.4,6.1,87.9,45.155728,5.762103,4,5
1819,2021-01-24,3.0,0.0,7.9,70.4,40.753680,0.813861,4,5
1820,2021-01-25,-1.2,0.0,6.2,75.8,33.588096,-3.487403,4,5


In [24]:
test_weather

,일시,기온,강수량,풍속,습도,불쾌지수,체감온도,불쾌지수구간,체감온도구간
0,2021-01-27,-1.2,0.0,6.6,61.5,35.802880,-3.646867,4,5
1,2021-01-28,-3.0,0.0,14.6,58.1,33.836130,-8.031683,4,5
2,2021-01-29,-6.0,0.0,11.0,39.4,33.465440,-10.787735,4,5
3,2021-01-30,-9.2,0.0,13.0,69.4,22.602848,-15.235758,4,5
4,2021-01-31,-6.0,1.0,6.0,74.6,26.340960,-8.922667,4,5
...,...,...,...,...,...,...,...,...,...
68,2021-04-05,4.5,0.0,6.6,58.9,44.146295,2.952329,4,5
69,2021-04-06,1.3,0.0,6.7,60.0,39.545200,-0.787847,4,5
70,2021-04-07,3.3,0.0,8.1,58.6,42.507662,1.109747,4,5
71,2021-04-08,5.5,0.0,6.4,64.1,45.078945,4.171122,4,5
